In [1]:
from functools import reduce
import math
import sys
import copy
import time
import random

In [2]:
not_use_pointer = 0
factorys_number = 0
factorys = []
customers_number = 0
customers = []
T = 1
data = None

In [3]:
def start(num):
    global not_use_pointer
    global factorys_number
    global factorys
    global customers_number
    global customers
    global T
    global data
    with open("Instances/p"+str(num),'r') as f:
        data = f.read().split()
        #print(data)
        for x in range(len(data)):
            if len(data[x])>10:
                data = data[:x]
                break
        def map_f(x):
            if x[-1]=='.':
                return int(x[:-1])
            else:
                return int(x)
        data = list(map(map_f,data))
    not_use_pointer = 0
    factorys_number = data[not_use_pointer]
    factorys = []
    not_use_pointer += 1
    customers_number = data[not_use_pointer]
    customers = []
    not_use_pointer += 1
    T = 10000
    init()

In [4]:
class Factoray:
    def __init__(self,capacity,open_cost,number):
        self.capacity = capacity
        self.open_cost = open_cost
        self.current_load = 0
        self.number = number
        self.assigned_customers = []
    def assign(self,customer):
        self.assigned_customers.append(customer.number)
        self.current_load += customer.load
    def unassign(self,customer):
        self.assigned_customers.pop(self.assigned_customers.index(customer.number))
        self.current_load -= customer.load
    def clean(self):
        self.curren_load = 0
        self.assigned_customers = []

In [5]:
class Customer:
    def __init__(self,load,distance_list,number):
        self.load = load
        self.distance_list = distance_list
        self.assigned_factory = None
        self.number = number
    def assign(self,_factory):
        if _factory.capacity - _factory.current_load < self.load:
            return False
        else:
            _factory.assign(self)
            self.assigned_factory = _factory.number
            return True
    def unassign(self):
        if self.assigned_factory != None:
            factorys[self.assigned_factory].unassign(self)
            self.assigned_factory = None
            return True
        return False

In [6]:
def init():
    global not_use_pointer
    global factorys_number
    global customers_number
    global factorys
    global customers
    global data
    for x in range(factorys_number):
        #print(not_use_pointer)
        factorys.append(Factoray(data[not_use_pointer],data[not_use_pointer+1],x))
        not_use_pointer += 2
    for x in range(customers_number):
        customers.append(Customer(data[not_use_pointer],[],x))
        not_use_pointer += 1
    for x in range(customers_number):
        for y in range(factorys_number):
            customers[x].distance_list.append(data[not_use_pointer])
            not_use_pointer+=1
    #print(not_use_pointer)
            #print(not_use_pointer)

In [7]:
def judge():
    global factorys
    global customers
    score = 0
    for factory in factorys:
        if factory.current_load != 0:
            score+=factory.open_cost
    for customer in customers:
        score+=customer.distance_list[customer.assigned_factory]
    return score
def judge_from(factorys,customers):
    #global factorys
    #global customers
    score = 0
    for factory in factorys:
        if factory.current_load != 0:
            score+=factory.open_cost
    for customer in customers:
        score+=customer.distance_list[customer.assigned_factory]
    return score

In [8]:
def find_n_least_big_index(arr,n,drop_factory_number):
    current_big = -1
    current_index_list = []
    for x in range(n):
        inner_big = 1000000
        inner_index = -1
        for y in range(len(arr)):
            if drop_factory_number != None:
                if y == drop_factory_number:
                    continue
            if y not in current_index_list:
                if arr[y] < inner_big:
                    inner_big = arr[y]
                    inner_index = y
        current_index_list.append(inner_index)
        if x == n-1:
            return inner_index

In [9]:
def gridy():
    global customers
    #print(customers)
    for customer in customers:
        for x in range(len(customer.distance_list)):
            target_index = find_n_least_big_index(customer.distance_list,x+1,None)
            if factorys[target_index].capacity - factorys[target_index].current_load < customer.load:
                continue
            else:
                customer.assign(factorys[target_index])
                break
def random_init():
    backoff()
    global customers
    #print(customers)
    for customer in customers:
        for x in range(len(customer.distance_list)):
            target_index = random.randint(0,len(customer.distance_list)-1)
            #target_index = find_n_least_big_index(customer.distance_list,x+1,None)
            if factorys[target_index].capacity - factorys[target_index].current_load < customer.load:
                continue
            else:
                customer.assign(factorys[target_index])
                break
    
def backoff():
    global customers
    global factorys
    for customer in customers:
        customer.unassign()
    for factory in factorys:
        factory.clean()
def compress_2_3():
    backoff()
    gridy()
    temp_min = judge()
    for x in range(len(factorys)-1):
        backoff()
        gridy()
        empty_facory_reassign(x)
        ## empty how many factory you want
        #empty_facory_reassign(x+1)
        if judge() < temp_min:
            temp_min = judge()
    print(temp_min)
def compress_2for2_3():
    backoff()
    gridy()
    temp_min = judge()
    for x in range(len(factorys)-2):
        ## empty how many factory you want
        #empty_facory_reassign(x+1)
        for y in range(x,len(factorys)-1):
            backoff()
            gridy()
            empty_facory_reassign(x)
            empty_facory_reassign(y)
            if judge() < temp_min:
                temp_min = judge()
    print(temp_min)
def empty_deep(empty_factory_list,start,end):
    temp_min = 1000000000
    if len(empty_factory_list) > 5:
        return temp_min
    for x in range(start,end):
        backoff()
        gridy()
        for index in empty_factory_list:
            empty_facory_reassign(index)
        empty_facory_reassign(x)
        temp_arr = empty_factory_list.copy()
        if judge() < temp_min:
            temp_min = judge()
        temp_arr.append(x)
        temp_value = empty_deep(temp_arr,x+1,end)
        if temp_value < temp_min:
            temp_min = temp_value
    return temp_min

In [10]:
def empty_facory_reassign(number):
    target_factory = factorys[number]
    target_factory.current_load = 0
    free_customers = target_factory.assigned_customers
    target_factory.assigned_customers = []
    for free_customer_number in free_customers:
        for x in range(len(customers[free_customer_number].distance_list)):
            target_index = find_n_least_big_index(customers[free_customer_number].distance_list,x+1,number)
            if factorys[target_index].capacity - factorys[target_index].current_load < customers[free_customer_number].load:
                continue
            else:
                customers[free_customer_number].assign(factorys[target_index])
                break

In [11]:
def possibility(energy):
    if energy <= 0:
        return 0
    global T
    #print(1 - 1/(1+math.e**(-energy/(T))))
    return 1 - 1/(1+math.e**(-energy/(T)))

In [12]:
def sa():
    global T
    while T>10**(-8):
        T = T*0.995
        for x in range(10):
            print(judge())
            change_customer_index = random.randint(0,len(customers)-1)
            #temp = judge()
            while True:
                ## move lots of customers at the same time
                #move_n_customers()
                change_to_factory_index = random.randint(0,len(factorys)-1)
                if factorys[change_to_factory_index].capacity - factorys[change_to_factory_index].current_load >= customers[change_customer_index].load:
                    #compare which one better
                    energy = 0
                    if factorys[customers[change_customer_index].assigned_factory].current_load == customers[change_customer_index].load:
                        energy -= customers[change_customer_index].load
                    energy += customers[change_customer_index].distance_list[change_to_factory_index]
                    energy -= customers[change_customer_index].distance_list[customers[change_customer_index].assigned_factory]
                    if energy<0 or possibility(energy) > random.random():
                        customers[change_customer_index].unassign()
                        customers[change_customer_index].assign(factorys[change_to_factory_index])
                    break

In [13]:
def sa2():
    global T
    global customers
    global factorys
    while T > 10**(-8):
        T *= 0.995
        for x in range(10):
            random_drop_number = random.randint(1,len(customers)//2)
            #print(len(customers))
            temp_arr = [x for x in range(len(customers))]
            random.shuffle(temp_arr)
            #temp_arr = random.shuffle(temp_arr)
            #print(temp_arr)
            temp_customers = copy.deepcopy(customers)
            temp_factorys = copy.deepcopy(factorys)
            temp_arr = temp_arr[:random_drop_number]
            #print(random_drop_number)
            for number in temp_arr:
                #temp_customers[number].unassign()
                #can not use them for temp object
                temp_factory = temp_customers[number].assigned_factory
                temp_customers[number].assigned_factory = None
                temp_factorys[temp_factory].current_load -= temp_customers[number].load
                temp_factorys[temp_factory].assigned_customers.pop(temp_factorys[temp_factory]\
                                                                   .assigned_customers.index(number))
            #print(temp_arr)
            random_start = random.randint(0,len(temp_arr)-1)
            for x in range(random_start,len(temp_arr)):
                x = temp_arr[x]
                customer = temp_customers[x]
                #for x in range(len(customer.distance_list)):
                while True:
                    #randomly
                    target_index = random.randint(0,len(customer.distance_list)-1)
                    #target_index = find_n_least_big_index(customer.distance_list,x+1,None)
                    if factorys[target_index].capacity - factorys[target_index].current_load < customer.load:
                        continue
                    else:
                        customer.assigned_factory = target_index
                        temp_factorys[target_index].current_load += customer.load
                        temp_factorys[target_index].assigned_customers.append(customer.number)
                        #customer.assign(factorys[target_index])
                    break
            for x in range(0,random_start):
                x = temp_arr[x]
                customer = temp_customers[x]
                #for x in range(len(customer.distance_list)):
                while True:
                    #randomly
                    target_index = random.randint(0,len(customer.distance_list)-1)
                    #target_index = find_n_least_big_index(customer.distance_list,x+1,None)
                    if factorys[target_index].capacity - factorys[target_index].current_load < customer.load:
                        continue
                    else:
                        customer.assigned_factory = target_index
                        temp_factorys[target_index].current_load += customer.load
                        temp_factorys[target_index].assigned_customers.append(customer.number)
                        #customer.assign(factorys[target_index])
                    break
            #print(possibility((judge()-judge_from(temp_factorys,temp_customers))))
            if judge_from(temp_factorys,temp_customers) < judge() or possibility(-(judge()-judge_from(temp_factorys,temp_customers))) > random.random():
                customers = temp_customers
                factorys = temp_factorys
                print(judge())
    return judge()

In [14]:
start(1)

In [15]:
gridy()
print(judge())

10355


In [16]:
#compress_2_3()
random_init()
judge()

25366

In [18]:
sa2()

10355

In [19]:
judge()

10355

In [ ]:
for x in range(1,10):
    print("___"+str(x)+"___")
    start(x)
    gridy()
    print(judge())
    print("+++")
    print(empty_deep([],0,len(factorys)))
    print("---")
    compress_2_3()

___1___
10355
+++
10070
---
10208
___2___
9041
+++
8908
---
8917
___3___
11041
+++
10508
---
10717
___4___
13041
+++
12108
---
12517
___5___
10827
+++


In [ ]:
start(20)
gridy()
judge()

In [ ]:
arr = [1,2,3]
print(arr.index(4,lambda x))

In [ ]:
a2 = [1,2,3]
a3 = a2.copy()

In [ ]:
a3.append(4)

In [ ]:
a = [Customer(1,[],0)]

In [ ]:
b = copy.copy(a)
b[0].number = 1

In [ ]:
a[0].number

In [ ]:
a = [1,2,3]
random.shuffle(a)


In [ ]:
a

In [4]:
for x in range(100):
    print(random.random())

0.3784218901670494
0.1969677475663869
0.18185931102388997
0.31282344388672867
0.38820167955446694
0.39704998460307184
0.7360730422767893
0.26230057923846295
0.23595370510133784
0.8137823979435758
0.4770614550649961
0.5047771004207079
0.3502238599829267
0.9806320214766455
0.8551282914835961
0.3792253839989388
0.8366368091763995
0.894305626383851
0.6239303669475598
0.7018330682752453
0.9098365299607282
0.4800322538128743
0.2976590712653231
0.7019178243819361
0.32676306174097935
0.48773270419469705
0.09447487139763189
0.8925279550814051
0.38371937595944416
0.870362679371342
0.6725026996246503
0.10939764761969506
0.5368079519768992
0.11754940778946132
0.07221322402861718
0.6612593361760417
0.724032850992958
0.45433976830216716
0.6766502243527848
0.9788397422138283
0.06939547622612074
0.22514684663115847
0.6633452853227203
0.5631043297969023
0.2836814431962401
0.2641565900293473
0.5154549193343873
0.812902681194003
0.3971069662068957
0.17049321004109086
0.47795760894929706
0.719125115121069